<a href="https://colab.research.google.com/github/inbox00/new/blob/main/torr%20%2B%20multiup%20mega.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install mega.py
!apt install python3-libtorrent

import libtorrent as lt
from google.colab import drive
from google.colab import files
import os
from mega import Mega

# Authenticate Mega
mega = Mega()
email = 'aaddkah@gmail.com'
password = 'Aa4465064'
m = mega.login(email, password)

# Create libtorrent session
ses = lt.session()
ses.listen_on(6881, 6891)
downloads = []

# Upload multiple torrent files
uploaded = files.upload()

# Parse and add torrents to session
for filename, torrent_content in uploaded.items():
    info = lt.torrent_info(torrent_content)
    print(f"Files in the torrent '{filename}':")
    for i, f in enumerate(info.files()):
        print(i, f.path)

    # Choose the index of the specific file you want to download or choose 'all'
    file_index = input(f"Enter the index of the file you want to download from '{filename}' or 'all' for all files: ")

    # Add the selected file(s) to the libtorrent session for downloading
    params = {"save_path": "/content/Torrent"}
    params["ti"] = info
    params["file_priorities"] = [1] * info.num_files() if file_index == 'all' else [0] * info.num_files()
    if file_index != 'all':
        file_index = int(file_index)
        params["file_priorities"][file_index] = 1

    downloads.append(ses.add_torrent(params))

# Monitor download progress
import time
from IPython.display import display
import ipywidgets as widgets

state_str = [
    "queued",
    "checking",
    "downloading metadata",
    "downloading",
    "finished",
    "seeding",
    "allocating",
    "checking fastresume",
]

layout = widgets.Layout(width="auto")
style = {"description_width": "initial"}
download_bars = [
    widgets.FloatSlider(
        step=0.01, disabled=True, layout=layout, style=style
    )
    for _ in downloads
]
display(*download_bars)

while downloads:
    next_shift = 0
    for index, download in enumerate(downloads[:]):
        bar = download_bars[index + next_shift]
        if not download.is_seed():
            s = download.status()

            bar.description = " ".join(
                [
                    download.name(),
                    str(s.download_rate / 1000),
                    "kB/s",
                    state_str[s.state],
                ]
            )
            bar.value = s.progress * 100
        else:
            next_shift -= 1
            ses.remove_torrent(download)
            downloads.remove(download)
            bar.close() # Seems to be not working in Colab (see https://github.com/googlecolab/colabtools/issues/726#issue-486731758)
            download_bars.remove(bar)
            print(download.name(), "complete")

            # Move downloaded file(s) to Mega
            if file_index == 'all':
                for file in info.files():
                    downloaded_file_path = os.path.join(params["save_path"], file.path)
                    m.upload(downloaded_file_path)
                    print("File uploaded to Mega:", os.path.basename(downloaded_file_path))
            else:
                downloaded_file_path = os.path.join(params["save_path"], info.files()[file_index].path)
                m.upload(downloaded_file_path)
                print("File uploaded to Mega:", os.path.basename(downloaded_file_path))

    time.sleep(1)


In [ ]:
from mega import Mega
import os
from tqdm import tqdm

# Function to ensure the directory exists
def ensure_dir(file_path):
    directory = os.path.dirname(file_path)
    if not os.path.exists(directory):
        os.makedirs(directory)

# Function to upload file with progress
def upload_file_with_progress(mega, file_path):
    file_size = os.path.getsize(file_path)
    with open(file_path, 'rb') as f:
        bar = tqdm(total=file_size, unit='B', unit_scale=True, desc=os.path.basename(file_path))
        # Reading in chunks and updating progress
        while True:
            chunk = f.read(1024 * 1024)  # Read in 1MB chunks
            if not chunk:
                break
            bar.update(len(chunk))
        bar.close()

# Authenticate with Mega
mega = Mega()
email = 'aaddkah@gmail.com'
password = 'Aa4465064'
m = mega.login(email, password)

# Directory and file paths
torrent_dir = '/content/Torrent/Renault Dialogys v4_72 EN'
files_to_check = [
    'Dialogys_v4_72_En_DVD4.iso',
    'Dialogys_v4_72_En_DVD5.iso',
    'Dialogys_v4_72_En_DVD6.iso',
    'Dialogys_v4_72_En_DVD7.iso'
]

# Ensure the directory exists
ensure_dir(torrent_dir)

for file_to_check in files_to_check:
    temp_file_path = os.path.join(torrent_dir, file_to_check)

    # Check if file exists and has correct size
    if os.path.exists(temp_file_path):
        file_size = os.path.getsize(temp_file_path)
        print(f"File found: {temp_file_path}")
        print(f"File size: {file_size} bytes")
    else:
        print(f"File not found: {temp_file_path}")
        file_size = 0

    # Only proceed to upload if the file size is correct
    if file_size > 0:
        print(f"Uploading {temp_file_path} to Mega...")
        upload_file_with_progress(m, temp_file_path)
        m.upload(temp_file_path)
        print("File uploaded successfully!")
    else:
        print("File size is incorrect, please check the file download process.")
